<a href="https://colab.research.google.com/github/pickledherring/NLP-group/blob/main/gloss_glove_sgns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from io import FileIO
from googleapiclient.http import MediaIoBaseDownload

import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

In [ ]:
#  Get the files from google drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Get english train data file
file_id = '1m3Ax9Z8OHMU-7FqraKc-ddI3YQ7yY_Q6'  # file id on the Google Drive
downloaded = FileIO("en.trial.complete.json", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))


Download 100%.


In [ ]:
en_df = pd.read_json("en.trial.complete.json")
en_df.head()

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
0,en.trial.1,beautiful,adjective,Pleasant ; clear .,"It 's beautiful outside , let 's go for a walk .",synonym/antonym-based,124908,706,0,0,"[1.393769145, 0.7516670227000001, -2.581333160...","[0.295645088, 0.098426342, 0.0463486575, 0.016...","[0.0800914839, -0.1875839084, -0.0411579385000..."
1,en.trial.2,cocktail,noun,A mixture of other substances or things .,a cocktail of illegal drugs,hypernym-based,4187,13245,1,0,"[2.0872907639, 0.2617726326, 0.668431639700000...","[0.3878918886, 0.1971583217, -0.44026631120000...","[-1.4771454334, -0.4742421806, 0.0847439319, -..."
2,en.trial.3,institutionalized,adjective,Having been established as an institution .,It is very difficult to get bureaucracies to a...,paraphrastic,961,35934,0,0,"[0.7893871069, -0.43510755900000003, 0.8553860...","[-0.0519028902, 0.2257766128, -0.1839749813, 0...","[-1.1030955315, -0.9046602845, 0.1503403783, -..."
3,en.trial.4,menial,noun,"A servant , especially a domestic servant .","The world was awake to the 2nd of May , but Ma...",hypernym-based,517,53267,1,1,"[0.1222261563, 0.1572209597, 0.5396134257, -0....","[-0.3667449057, -0.1431699395, -0.0671329796, ...","[-1.6584062576, -0.24498166140000002, 0.150174..."
4,en.trial.5,seek,verb,To try to find ; to look for ; to search for .,"Not long ago , it was difficult to produce pho...",paraphrastic,25195,3212,0,0,"[1.1894155741, 1.3668279648000001, -1.61634504...","[0.6137102246, 0.5464909673, -0.0161557049, 9....","[-0.5474479198000001, -0.0880863219, 0.0784259..."


In [ ]:
en_df.dtypes

id            object
word          object
pos           object
gloss         object
example       object
type          object
counts         int64
f_rnk          int64
concrete       int64
polysemous     int64
sgns          object
char          object
electra       object
dtype: object

In [ ]:
en_df[en_df.word.duplicated()]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


we have words to disambiguate! forthwith looks to be the same with a different example though. <br>thoughts on disambiguation (feel free to add):<br><br> need to gather contex, but all I can think of is to train the NN to find the part of speech of the word and maybe type. I think we are only given the gloss and from that predict the embedding, so we (maybe) can't use the rest of it except in training. probably a good idea regardless of the disambiguation goal. <br><br> we could also retrain the model on incorrect words that are polysemous, like a boosting method. <br><br> lastly, disambiguation might not be necessary given the gloss and the vector might not be so similar. should look at the distance between these words in the embeddings. glosses look different, but I can see the top few elements of these two polysemous words are similar in all of the embeddings.

In [ ]:
en_df[(en_df.word == "divorce") | (en_df.word == "forthwith")]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
50,en.trial.51,divorce,verb,To separate something that was connected .,The radical group voted to divorce itself from...,hypernym-based,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[-0.6520454288, -0.1928912848, 0.0298318155, -..."
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
160,en.trial.161,forthwith,adverb,Without delay ; immediately .,"Let ther be Light , said God , and forthwith L...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-0.9691627026, -0.20675842460000002, 0.184671..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


clean glosses of punctuation, turn into lists. no stopwords removed due to small size of gloss, possible future addition to "clean". probably should remove duplicates, too.

In [ ]:
def clean(gloss):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  cleaned = tokenizer.tokenize(gloss)
  cleaned = [word.lower() for word in cleaned]
  return cleaned

gloss_lists = en_df.gloss.apply(clean)
gloss_lists

0                                      [pleasant, clear]
1        [a, mixture, of, other, substances, or, things]
2       [having, been, established, as, an, institution]
3         [a, servant, especially, a, domestic, servant]
4      [to, try, to, find, to, look, for, to, search,...
                             ...                        
195           [any, color, in, plant, or, animal, cells]
196                                [appearance, seeming]
197    [a, permission, for, a, vehicle, to, proceed, ...
198                 [not, moving, much, sitting, around]
199    [a, sentence, phrase, or, word, forming, part,...
Name: gloss, Length: 200, dtype: object

is using pretrained glove vectors cheating? should we be generating our own glove vectors from the training data?

In [ ]:
# Get the GLoVe vectors file, obtained from https://nlp.stanford.edu/projects/glove/
# specifically, the Wikipedia 2014 + Gigaword 5 dataset, 50d
file_id = '1w3-Ds25h6BPjqP_NaTOHS-jU-bZBCPDp'
downloaded = FileIO("glove_50d.txt", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))

Download 61%.
Download 100%.


In [ ]:
# taken from https://stackoverflow.com/a/38230349
def load_glove_model(File):
  glove_model = {}
  with open(File,'r') as f:
    for line in f:
      split_line = line.split()
      word = split_line[0]
      glove_model[word] = [float(x) for x in split_line[1:]]
  return glove_model

glove_words = load_glove("glove_50d.txt")

function to make word embedding for each word in the gloss, then average those embeddings. averaging might not be the best approach, just the easiest.

In [ ]:
def get_glove(gloss_list):
  length = len(glove_words[next(iter(glove_words))])
  glove_avg = [0] * length

  for word in gloss_list:
    if word in glove_words:
      for i in range(length):
        glove_avg[i] += float(glove_words[word][i])

  return [x / len(gloss_list) for x in glove_avg]

vectors = gloss_lists.apply(get_glove)

In [ ]:
vectors[:5]

0    [0.52813, 0.748455, -0.46263, -0.25607, 0.4127...
1    [0.3234542857142858, 0.09864314285714286, -0.4...
2    [0.3238285, 0.09639766666666666, -0.3500858333...
3    [-0.015297833333333344, 0.19006800000000001, -...
4    [0.6287290000000001, 0.008221399999999995, 0.1...
Name: gloss, dtype: object

In [ ]:
y = en_df.sgns
X_train, X_test, y_train, y_test = train_test_split(vectors, y)

next up: scale the data, transform it to the right dimensions, build a simple neural network, fit it with the train data, see if we can get sgns vectors, compare with cosine similarity to the test